# SÉANCE 1

### ÉTAPE 2 - Persona

#### Métier / Type d’utilisateur intéressé par ces données
Un thérapeute de couple ou un psychologue conjugal serait un profil très pertinent. Ces professionnels cherchent à comprendre les facteurs influençant la satisfaction conjugale et la fidélité dans le couple.

Les variables disponibles (âge, durée du mariage, présence d’enfants, religiosité, satisfaction dans le couple, etc.) peuvent les aider à identifier les profils à risque ou à mieux orienter leur accompagnement.

#### Persona : Dr. Clara Morel, psychologue conjugale
__Âge :__ 42 ans

__Profession :__ Psychologue spécialisée en thérapie de couple

__Lieu d’exercice :__ Cabinet privé à Lyon

__Contexte :__ Elle accompagne principalement des couples mariés entre 25 et 55 ans. Elle s’intéresse à l’impact de la durée du mariage, de la religiosité, du niveau d’éducation ou encore du niveau de satisfaction conjugale sur la fidélité.

### ÉTAPE 3 - Chargement, pré-traitement

In [11]:
import org.apache.spark.rdd.RDD

In [15]:
case class Affair(
    id: Int,
    affairs: Int,
    gender: String,
    age: Double,
    yearsMarried: Double,
    children: String,
    religiousness: Int,
    education: Int,
    occupation: Int,
    rating: Int
)

defined class Affair


In [14]:
val data = sc.textFile("data/Affairs.csv")
    .zipWithIndex().filter { case (_, idx) => idx > 0 }.map(_._1)

val affairsRDD: RDD[Affair] = data
  .map(_.split(","))
  .map(fields => Affair(
    fields(0).toInt,
    fields(1).toInt,
    fields(2).toString,
    fields(3).toDouble,
    fields(4).toDouble,
    fields(5).toString,
    fields(6).toInt,
    fields(7).toInt,
    fields(8).toInt,
    fields(9).toInt
  ))

affairsRDD.take(10).foreach(println)

Affair(4,0,male,37.0,10.0,no,3,18,7,4)
Affair(5,0,female,27.0,4.0,no,4,14,6,4)
Affair(11,0,female,32.0,15.0,yes,1,12,1,4)
Affair(16,0,male,57.0,15.0,yes,5,18,6,5)
Affair(23,0,male,22.0,0.75,no,2,17,6,3)
Affair(29,0,female,32.0,1.5,no,2,17,5,5)
Affair(44,0,female,22.0,0.75,no,2,12,1,3)
Affair(45,0,male,57.0,15.0,yes,2,14,4,4)
Affair(47,0,female,32.0,15.0,yes,4,16,1,2)
Affair(49,0,male,22.0,1.5,no,4,14,4,5)


data = MapPartitionsRDD[32] at map at <console>:36
affairsRDD = MapPartitionsRDD[34] at map at <console>:40


MapPartitionsRDD[34] at map at <console>:40